# READ ONLY

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.tabular import *
from utils import *

from sklearn.metrics import cohen_kappa_score,confusion_matrix,mean_squared_error,accuracy_score,make_scorer

from fastai.callbacks.tracker import *

path = Path.cwd()/'data/'

In [3]:
from fastai.text import *
from fastai.text.data import _join_texts

In [4]:
def reset_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#     tf.set_random_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
reset_seed()

In [5]:
train,test = preprocess_data_tabular(for_nn=True,drop_description=False)

X_train shape: (14990, 151)
X_test shape: (3948, 149)


In [6]:
train.columns[train.isna().any()].tolist()

['sentiment_document_magnitude',
 'sentiment_magnitude',
 'sentiment_document_score',
 'sentiment_score',
 'metadata_metadata_annots_score_MEAN',
 'metadata_metadata_annots_score_SUM',
 'metadata_metadata_color_score_MEAN',
 'metadata_metadata_color_score_SUM',
 'metadata_metadata_color_pixelfrac_MEAN',
 'metadata_metadata_color_pixelfrac_SUM',
 'metadata_metadata_crop_conf_MEAN',
 'metadata_metadata_crop_conf_SUM',
 'metadata_metadata_crop_importance_MEAN',
 'metadata_metadata_crop_importance_SUM',
 'image_size_sum',
 'image_size_mean',
 'image_size_std',
 'width_sum',
 'width_mean',
 'width_std',
 'height_sum',
 'height_mean',
 'height_std']

In [7]:
def generate_val_idxs():
    seed=42
    np.random.seed(seed)
    rIDs = np.random.permutation(train.RescuerID.unique()).tolist()
    
    val_len = int(.2 * len(train.RescuerID.unique()) + 1)
    train_idxs=[]
    val_idxs=[]
    for i in range(5):
        val_rIDs = rIDs[val_len*i:val_len*(i+1)]
        train_rIDs=rIDs[0:val_len*i] + rIDs[val_len*(i+1):]
        train_idxs.append(train[train.RescuerID.isin(train_rIDs)].index.values)
        val_idxs.append(train[train.RescuerID.isin(val_rIDs)].index.values)
    return train_idxs,val_idxs

In [8]:
train_idxs,val_idxs = generate_val_idxs()
val_idxs

[array([    8,    13,    16,    19, ..., 14973, 14978, 14985, 14989]),
 array([   15,    21,    26,    27, ..., 14963, 14971, 14983, 14984]),
 array([    0,     6,     7,    24, ..., 14967, 14975, 14982, 14986]),
 array([    1,     2,    11,    12, ..., 14976, 14980, 14981, 14988]),
 array([    3,     4,     5,     9, ..., 14968, 14977, 14979, 14987])]

In [9]:
train,test=mean_encoding(train,test,train_idxs,val_idxs,for_nn=True)
print(train.shape)
print(test.shape)

(14990, 167)
(3948, 165)


In [10]:
train=train.drop(['RescuerID'],axis=1)
print(train.shape)
print(test.shape)

(14990, 166)
(3948, 165)


In [11]:
cat_names=['Type','Breed1', 'Breed2','Gender','Color1', 'Color2', 'Color3',
          'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed','Sterilized', 'Health','State','m1_label_description',
           'Is_Sterilized','Is_Free','IsMandarin','IsMalay']
# cont_names= list(set(train.columns) - set(cat_names) - {'AdoptionSpeed'})
cont_names= list(set(train.columns) - set(cat_names) - {'AdoptionSpeed','Description'})
print(f'# of continuous feas: {len(cont_names)}')
print(f'# of categorical feas: {len(cat_names)}')
dep_var = 'AdoptionSpeed'
procs = [FillMissing,Categorify, Normalize]

# of continuous feas: 145
# of categorical feas: 19


In [12]:
txt_cols=['Description']

In [13]:
len(cat_names) + len(cont_names) + 2 == train.shape[1]

True

In [14]:
def get_learner(data,layers,save_name='best_nn',emb_drop=0.0,ps=None):
    return tabular_learner(data, layers=layers, metrics=[root_mean_squared_error],emb_drop=emb_drop,ps=ps,
#                         callback_fns=[partial(EarlyStoppingCallback, monitor='quadratic_kappa_score',mode='max', min_delta=0.01, patience=3)]
                           callback_fns=[partial(SaveModelCallback, monitor='root_mean_squared_error',mode='min',every='improvement',name=save_name)],
                           
                          )
def get_learner_no_cb(data,layers,emb_drop=0.0,ps=None):
    return tabular_learner(data, layers=layers, emb_drop=emb_drop,ps=ps,metrics=[root_mean_squared_error])


# Get tabular databunch

In [16]:
def get_databunch(k=0,bs=64,val_idxs=val_idxs):
    reset_seed()
    data = (TabularList.from_df(train, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_idxs[k])
                           .label_from_df(cols=dep_var)
                           .add_test(TabularList.from_df(test, path=path, cat_names=cat_names, cont_names=cont_names))
                           .databunch(bs=bs))
    return data

l2str= lambda layers: '_'.join([str(l) for l in layers])

# Get text databunch

In [17]:
def get_text_databunch(bs=64):
    X_train = train[['Description','AdoptionSpeed']].copy()
    X_test = test['Description'].copy()
    
    data_lm = load_data(path, 'tmp_lm.pkl', bs=bs)
    

    X_train.AdoptionSpeed = X_train.AdoptionSpeed.astype('int32')
    
    reset_seed()
    data_clas = (TextList.from_df(X_train,path, vocab=data_lm.vocab)
                 #grab all the text files in path
                 .split_by_idx(val_idxs[0])
                 #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
                 .label_from_df(cols='AdoptionSpeed',label_cls=FloatList)
                 #label them all with their folders
                 .add_test(TabularList.from_df(X_test, path=path))
                 .databunch(bs=bs))

    # data_clas.save('data_clas.pkl')
    return data_clas
#     return load_data(path, 'data_clas.pkl', bs=32)

# Start experiment

In [18]:
import pdb

https://github.com/ohmeow/medium-finding-data-block-nirvana/blob/master/yelp-00-custom-itemlist.ipynb

Medium:

https://blog.usejournal.com/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-c38210537fe4

## Complete code block here

In [19]:
class TabularText(TabularLine):
    "Item's that include both tabular data(`conts` and `cats`) and textual data (numericalized `ids`)"
    
    def __init__(self, cats, conts, cat_classes, col_names, txt_ids, txt_cols, txt_string):
        # tabular
        super().__init__(cats, conts, cat_classes, col_names)

        # add the text bits
        self.text_ids = txt_ids
        self.text_cols = txt_cols
        self.text = txt_string
        
        # append numericalted text data to your input (represents your X values that are fed into your model)
        # self.data = [tensor(cats), tensor(conts), tensor(txt_ids)]
        self.data += [ np.array(txt_ids, dtype=np.int64) ]
        self.obj = self.data
        
    def __str__(self):
        res = super().__str__() + f'Text: {self.text}'
        return res

class TabularTextProcessor(TabularProcessor):
    # The processors are called at the end of the labelling to apply some kind of function on your items. 
    # The default processor of the inputs can be overriden by passing a processor in the kwargs when creating the ItemList, 
    # the default processor of the targets can be overriden by passing a processor in the kwargs of the labelling function.
    def __init__(self, ds:ItemList=None, procs=None, 
                 #tokenize processor args
                 tokenizer:Tokenizer=None, chunksize:int=10000, # no mark_fields
                 include_bos:bool=True, include_eos:bool=False, #include_bos, include_eos for def proces
                 
                 # numericalize processor args
                 vocab:Vocab=None, max_vocab:int=60000, min_freq:int=2):
        super().__init__(ds, procs)
        
        #tokenize
        self.tokenizer, self.chunksize = ifnone(tokenizer, Tokenizer()), chunksize
        self.include_bos, self.include_eos = include_bos, include_eos
        
        #numericalize
        vocab = ifnone(vocab, ds.vocab if ds is not None else None)
        self.vocab, self.max_vocab, self.min_freq = vocab, max_vocab, min_freq
        
    # process a single item in a dataset
    # NOTE: THIS IS METHOD HAS NOT BEEN TESTED AT THIS POINT (WILL COVER IN A FUTURE ARTICLE)
    def process_one(self, item):
#         # process tabular data (copied form tabular.data)
#         df = pd.DataFrame([item, item])
#         for proc in self.procs: proc(df, test=True)
            
#         if len(self.cat_names) != 0:
#             codes = np.stack([c.cat.codes.values for n,c in df[self.cat_names].items()], 1).astype(np.int64) + 1
#         else: 
#             codes = [[]]
            
#         if len(self.cont_names) != 0:
#             conts = np.stack([c.astype('float32').values for n,c in df[self.cont_names].items()], 1)
#         else: 
#             conts = [[]]
            
#         classes = None
#         col_names = list(df[self.cat_names].columns.values) + list(df[self.cont_names].columns.values)
        
#         # process textual data
#         if len(self.text_cols) != 0:
#             txt = _join_texts(df[self.text_cols].values, (len(self.text_cols) > 1))
#             txt_toks = self.tokenizer._process_all_1(txt)[0]
#             text_ids = np.array(self.vocab.numericalize(txt_toks), dtype=np.int64)
#         else:
#             txt_toks, text_ids = None, [[]]
            
#         # return ItemBase
#         return MixedTabularLine(codes[0], conts[0], classes, col_names, text_ids, self.txt_cols, txt_toks)
        pass
    # processes the entire dataset
    def process(self, ds):
        '''
        ds is itembase
        '''
        # process tabular data and then set "preprocessed=False" since we still have text data possibly
        super().process(ds)
        ds.preprocessed = False
        
        # process text data from column(s) containing text
        if len(ds.text_cols) != 0:
            texts = _join_texts(ds.inner_df[ds.text_cols].values, (len(ds.text_cols) > 1), self.include_bos, self.include_eos)

            # tokenize (set = .text)
            tokens = []
            for i in progress_bar(range(0, len(ds), self.chunksize), leave=False):
                tokens += self.tokenizer.process_all(texts[i:i+self.chunksize])
            ds.text = tokens
            
            # numericalize 
            # set/build vocab
            if self.vocab is None: self.vocab = Vocab.create(ds.text, self.max_vocab, self.min_freq)
            ds.vocab = self.vocab
            ds.text_ids = [ np.array(self.vocab.numericalize(toks), dtype=np.int64) for toks in ds.text ]
        else:
            ds.text, ds.vocab, ds.text_ids = None, None, []
            
        ds.preprocessed = True
        
        
def mixed_tabular_pad_collate(samples:BatchSamples, 
                              pad_idx:int=1, pad_first:bool=True) -> Tuple[LongTensor, LongTensor]:
    "Function that collect samples and adds padding."
    # we need to add padding to the column with the text ids in order to ensure 
    # a square matrix per batch before integrating the text bits with the tabular.
    

    samples = to_data(samples)
    max_len = max([len(s[0][-1]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
   
    for i,s in enumerate(samples):
        if pad_first: 
            res[i,-len(s[0][-1]):] = LongTensor(s[0][-1])
        else:         
            res[i,:len(s[0][-1]):] = LongTensor(s[0][-1])
            
        # replace the text_ids array (the last thing in the inputs) with the padded tensor matrix
        s[0][-1] = res[i]
    

    # for the inputs, return a list containing 3 elements: a list of cats, a list of conts, and a list of text_ids
    # also include tensor list of classes
    return [torch.stack(x) for x in zip(*[s[0] for s in samples])],tensor(np.array([s[1] for s in samples]))


# each "ds" is of type LabelList(Dataset)
class TabularTextDataBunch(DataBunch):
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs=64, 
               pad_idx=1, pad_first=True, no_check:bool=False, **kwargs) -> DataBunch:
        
        # only thing we're doing here is setting the collate_fn = to our new "pad_collate" method above
        collate_fn = partial(mixed_tabular_pad_collate, pad_idx=pad_idx, pad_first=pad_first)
        
        return super().create(train_ds, valid_ds, test_ds, path=path, bs=bs,**kwargs)

    
    
class TabularTextList(TabularList):
    "A custom `ItemList` that merges tabular data along with textual data"
    
    _item_cls = TabularText
    _processor = TabularTextProcessor
    _bunch = TabularTextDataBunch
    
    def __init__(self, items:Iterator, cat_names:OptStrList=None, cont_names:OptStrList=None, 
                 text_cols=None, vocab:Vocab=None, pad_idx:int=1, 
                 procs=None, **kwargs) -> 'MixedTabularList':
        super().__init__(items, cat_names, cont_names, procs, **kwargs)
        
        self.cols = [] if cat_names == None else cat_names.copy()
        if cont_names: self.cols += cont_names.copy()
        if txt_cols: self.cols += text_cols.copy()
        
        # from TextList
        self.text_cols, self.vocab, self.pad_idx = text_cols, vocab, pad_idx
        
        # add any ItemList state into "copy_new" that needs to be copied each time "new()" is called; 
        # your ItemList acts as a prototype for training, validation, and/or test ItemList instances that
        # are created via ItemList.new()
        self.copy_new += ['text_cols', 'vocab', 'pad_idx']
        
        self.preprocessed = False
        
    # defines how to construct an ItemBase from the data in the ItemList.items array
    def get(self, i):
        if not self.preprocessed: 
            return self.inner_df.iloc[i][self.cols] if hasattr(self, 'inner_df') else self.items[i]
        
        codes = [] if self.codes is None else self.codes[i]
        conts = [] if self.conts is None else self.conts[i]
        
        #from TextList
        text_ids = [] if self.text_ids is None else self.text_ids[i]
        text_string = None if self.text_ids is None else self.vocab.textify(self.text_ids[i])
        
        return self._item_cls(codes, conts, self.classes, self.col_names, text_ids, self.text_cols, text_string)
    
    # this is the method that is called in data.show_batch(), learn.predict() or learn.show_results() 
    # to transform a pytorch tensor back in an ItemBase. 
    # in a way, it does the opposite of calling ItemBase.data. It should take a tensor t and return 
    # the same kind of thing as the get method.
    def reconstruct(self, t:Tensor):
        # TODO?
        idx_min = (t[2] != self.pad_idx).nonzero().min()
        idx_max = (t[2] != self.pad_idx).nonzero().max()
        return self._item_cls(t[0], t[1], self.classes, self.col_names, 
                              t[2][idx_min:idx_max], self.text_cols, self.vocab.textify(t[2][idx_min:idx_max]))
        
#         return self._item_cls(t[0], t[1], self.classes, self.col_names, 
#                               t[2], self.text_cols, self.vocab.textify(t[2])) 
    

        
        
    # tells fastai how to display a custom ItemBase when data.show_batch() is called
    def show_xys(self, xs, ys) -> None:
        "Show the `xs` (inputs) and `ys` (targets)."
        from IPython.display import display, HTML
        
        # show tabular
        display(HTML('TABULAR:<br>'))
        super().show_xys(xs, ys)
        
        # show text        
        display(HTML('TEXT:<br>'))        
        names = ['text', 'target']
        items = []
        for i, (x,y) in enumerate(zip(xs,ys)):
            res = []
            res.append(' '.join([ f'{tok}({self.vocab.stoi[tok]})' 
                              for tok in x.text.split() if (not self.vocab.stoi[tok] == self.pad_idx) ]))
                
            res.append(str(y))
            items.append(res)

        items = np.array(items)
        df = pd.DataFrame({n:items[:,i] for i,n in enumerate(names)})
        with pd.option_context('display.max_colwidth', -1):
            display(HTML(df.to_html(index=False)))
        
    # tells fastai how to display a custom ItemBase when learn.show_results() is called
    def show_xyzs(self, xs, ys, zs):
        "Show `xs` (inputs), `ys` (targets) and `zs` (predictions)."
        from IPython.display import display, HTML
        
        # show tabular
        display(HTML('TABULAR:<br>'))
        super().show_xyzs(xs, ys, zs)
        
        # show text        
        display(HTML('TEXT:<br>'))        
        names = ['text', 'target','pred']
        items = []
        for i, (x,y,z) in enumerate(zip(xs,ys,zs)):
            res = []
            res.append(' '.join([ f'{tok}({self.vocab.stoi[tok]})' 
                              for tok in x.text.split() if (not self.vocab.stoi[tok] == self.pad_idx) ]))
                
            res += [str(y),str(z)]
            items.append(res)
            
        items = np.array(items)
        df = pd.DataFrame({n:items[:,i] for i,n in enumerate(names)})
        with pd.option_context('display.max_colwidth', -1):
            display(HTML(df.to_html(index=False)))
    
        
    @classmethod
    def from_df(cls, df:DataFrame, cat_names:OptStrList=None, cont_names:OptStrList=None, 
                text_cols=None, vocab=None, procs=None, **kwargs) -> 'ItemList':
        
        return cls(items=range(len(df)), cat_names=cat_names, cont_names=cont_names, 
                   text_cols=text_cols, vocab=vocab, procs=procs, inner_df=df.copy(), **kwargs)

In [20]:
def get_tabulartext_databunch(k=0,bs=64,val_idxs=val_idxs):
    data_lm = load_data(path, 'tmp_lm.pkl', bs=bs)
    collate_fn = partial(mixed_tabular_pad_collate, pad_idx=1, pad_first=True)
    reset_seed()
    return (TabularTextList.from_df(train, cat_names, cont_names, txt_cols, vocab=data_lm.vocab, procs=procs, path=path)
                            .split_by_idx(val_idxs[k])
                            .label_from_df(cols=dep_var)
                            .add_test(TabularTextList.from_df(test, cat_names, cont_names, txt_cols,path=path))
                            .databunch(bs=bs,collate_fn=collate_fn, no_check=False))

# Tab and text learner

In [22]:
# def tabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
#         ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
#     "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
#     emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
#     model = TabularModel(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
#                          y_range=y_range, use_bn=use_bn)
#     return Learner(data, model, metrics=metrics, **learn_kwargs)


In [67]:
class PoolingLinearTabularTextClassifier(nn.Module):
    "Create a linear classifier with pooling."

    def __init__(self, rnn_lin_layers:Collection[int], ps_lin_ftrs:Collection[float],
                 # tabular params inputs
                 emb_szs,n_cont,n_class,layers,ps,emb_drop,y_range,use_bn,bn_final):
        # rnn_lin_layers: [1200, 50, 1]
        # ps_lin_ftrs: [0.4 (from output_p, layer1200), 0.1 for layer50]
        
        super().__init__()
        # text layers
        mod_layers = []
        activs = [nn.ReLU(inplace=True)] * (len(rnn_lin_layers) - 2) + [None]
        for n_in,n_out,p,actn in zip(rnn_lin_layers[:-1],rnn_lin_layers[1:], ps_lin_ftrs, activs):
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
        mod_layers = mod_layers[:-1] # exclude the last linear output
#         self.layers = nn.Sequential(*mod_layers[:-1]) 
    
    
        #tabular layers
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
                
        # embedding stuff
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop) # drop for embedding
        self.bn_cont = nn.BatchNorm1d(n_cont) # bn for continuous features
        n_emb = sum(e.embedding_dim for e in self.embeds) # total length of cat embeddings
        
               
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, rnn_lin_layers[-2],n_class) # [343, 222, 111, 1] ->convert to [343+ 50, 222, 111, 1]        
        actns = [nn.ReLU(inplace=True)] * (len(sizes)-2) + [None] # [ReLU(inplace), ReLU(inplace), None]
        
        # self.layers can stay the same
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        
#         print(mod_layers + layers)
        self.layers = nn.Sequential(*(mod_layers + layers))

        
        
    def get_sizes(self, layers, rnn_lin_layer, out_sz):
        # concatenate cat,conts of tabular and rnn lin layer 
        return [self.n_emb + self.n_cont + rnn_lin_layer] + layers + [out_sz]
    
#     def forward(self, input:Tuple[Tensor,Tensor, Tensor])->Tuple[Tensor,Tensor,Tensor]:
#         pdb.set_trace()
#         raw_outputs,outputs,mask = input
#         output = outputs[-1]
#         avg_pool = output.masked_fill(mask[:,:,None], 0).mean(dim=1)
#         avg_pool *= output.size(1) / (output.size(1)-mask.float().sum(dim=1))[:,None]
#         max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
#         x = torch.cat([output[:,-1], max_pool, avg_pool], 1)
#         x = self.layers(x)
#         return x, raw_outputs, outputs # TODO: why do we need raw_outputs and outputs?
#         return None
    
    def forward(self, input:Tuple[Tensor,Tensor,Tensor,Tensor,Tensor]):
        x_cat,x_cont,raw_outputs,outputs,mask = input
        # text
        output = outputs[-1]
        avg_pool = output.masked_fill(mask[:,:,None], 0).mean(dim=1)
        avg_pool *= output.size(1) / (output.size(1)-mask.float().sum(dim=1))[:,None]
        max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
        x_text = torch.cat([output[:,-1], max_pool, avg_pool], 1) #(bs,1200) for AWD LSTM
        
        # tabular
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
            
        x = torch.cat([x_text,x],1)
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x,raw_outputs,outputs # TODO: why do we need raw_outputs and outputs?

class MultiBatchMixEncoder(MultiBatchEncoder):
    "Create an encoder over `module` that can process a full sentence."
    def __init__(self, bptt:int, max_len:int, module:nn.Module, pad_idx:int=1):
        super().__init__(bptt,max_len,module,pad_idx)

    def forward(self, x_cat:Tensor,x_cont:Tensor,x_text:LongTensor):
        bs,sl = x_text.size()
        self.reset()
        raw_outputs,outputs,masks = [],[],[]
        for i in range(0, sl, self.bptt):
            r, o = self.module(input[:,i: min(i+self.bptt, sl)])
            if i>(sl-self.max_len):
                masks.append(input[:,i: min(i+self.bptt, sl)] == self.pad_idx)
                raw_outputs.append(r)
                outputs.append(o)
        return x_cat,x_cont,self.concat(raw_outputs),self.concat(outputs),torch.cat(masks,dim=1)

                                
def get_tabular_text_classifier(emb_szs:ListSizes, n_cont:int , n_class:int, layers:Collection[int], 
                                # text classifier params inputs
                                arch:Callable, vocab_sz:int, bptt:int=70, max_len:int=20*70, config:dict=None, 
                                drop_mult:float=1., lin_ftrs:Collection[int]=None, ps_lin_ftrs:Collection[float]=None,pad_idx:int=1,
                                # tabular params inputs
                                ps:Collection[float]=None,emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False
                               ) -> nn.Module:
    "Create a text classifier from `arch` and its `config`, maybe `pretrained`."
    
    # TODO: remove text.learner.*?
    meta = text.learner._model_meta[arch]
    config = ifnone(config, meta['config_clas'].copy())
    for k in config.keys(): 
        if k.endswith('_p'): config[k] *= drop_mult # drop_mult: multiply to different dropouts in AWD LSTM
    if lin_ftrs is None: lin_ftrs = [50]
    if ps_lin_ftrs is None:  ps_lin_ftrs = [0.1]
    
    rnn_lin_layers = [config[meta['hid_name']] * 3] + lin_ftrs + [n_class] # [1200, 50, 1]
    ps_lin_ftrs = [config.pop('output_p')] + ps_lin_ftrs #[0.4 (from output_p), 0.1 for layer50]
    init = config.pop('init') if 'init' in config else None
    encoder = MultiBatchMixEncoder(bptt, max_len, arch(vocab_sz, **config), pad_idx=pad_idx)
    
    tabtext_lin_model = PoolingLinearTabularTextClassifier(rnn_lin_layers, ps_lin_ftrs,
                                                       # tabular params inputs
                                                      emb_szs,n_cont,n_class,layers,ps,emb_drop,y_range,use_bn,bn_final)
    final_model = SequentialRNN(encoder, tabtext_lin_model)
    return final_model if init is None else final_model.apply(init)

In [68]:
def tabtext_learner(data,layers:Collection[int], 
                    arch:Callable,
                    # text classifier params inputs
                    metrics=None,
                    bptt:int=70, max_len:int=20*70, config:dict=None,                     
                    drop_mult:float=1., lin_ftrs:Collection[int]=None, ps_lin_ftrs:Collection[float]=None,pad_idx:int=1,
                    # tabular params inputs
                    emb_szs:Dict[str,int]=None, ps:Collection[float]=None,emb_drop:float=0., 
                    y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False,pretrained:bool=True, **learn_kwargs):
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = get_tabular_text_classifier(emb_szs,len(data.cont_names),data.c,layers,
                                       arch,len(data.vocab.itos),bptt=bptt,max_len=max_len,config=config,
                                        drop_mult=drop_mult, lin_ftrs=lin_ftrs,ps_lin_ftrs=ps_lin_ftrs,
                                        pad_idx=pad_idx,ps=ps,emb_drop=emb_drop,y_range=y_range,
                                       use_bn=use_bn,bn_final=bn_final)
    #text
    meta = text.learner._model_meta[arch]
    learn = RNNLearner(data, model, metrics = metrics, split_func=meta['split_clas'], **learn_kwargs)
    if pretrained:
        if 'url' not in meta: 
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], data=False)
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn.load_pretrained(*fnames, strict=False)
        learn.freeze()
    return learn

#     learn = Learner(data,model,metrics=metrics,**learn_kwargs) 

In [57]:
tabtext_db = get_tabulartext_databunch()

In [61]:
learn = tabtext_learner(tabtext_db,[222,111],AWD_LSTM,metrics=[root_mean_squared_error],
                       callback_fns=[partial(SaveModelCallback, monitor='root_mean_squared_error',mode='min',every='improvement',name='best_nn')])

In [70]:
learn.load_encoder('bs256-8e-awdlstm-enc')

In [69]:
learn.callback_fns

[functools.partial(<class 'fastai.basic_train.Recorder'>, add_time=True),
 functools.partial(<class 'fastai.callbacks.tracker.SaveModelCallback'>, monitor='root_mean_squared_error', mode='min', every='improvement', name='best_nn')]

In [38]:
# emb_szs = tabtext_db.get_emb_szs({})
# model = get_tabular_text_classifier(emb_szs,len(tabtext_db.cont_names),tabtext_db.c,[222,111],
#                                    AWD_LSTM,len(tabtext_db.vocab.itos),lin_ftrs=[55,44],ps_lin_ftrs=[0.55,0.44],ps=[0.22,0.11],
#                                    emb_drop=0.111)

In [39]:
# model

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(10131, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(10131, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1150, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1150, 1150, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1150, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearTabTextClassifier(
    (embeds): ModuleList(
      (0): Embedding(3, 3)
      (1): Embedding(165, 28)
      (2): Embedding(123, 24)
      (3): Embedding(4, 3)
      (4): Embedding(8, 5)
      (5): Embedding(8, 5)
      (6): Embedding(7, 5)
      (7): Embedding(5, 4)
      (8): Embedding(4, 3)

In [97]:
# model = get_text_classifier(AWD_LSTM, len(tabtext_db.vocab.itos), tabtext_db.c, bptt=70, max_len=70*20,
#                                 drop_mult=1.)

In [98]:
# model

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(10131, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(10131, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1150, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1150, 1150, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1150, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.4)
      (2): Linear(in_features=1200, out_features=50, bias=True)
      (3): ReLU(inplace)
      (4): BatchNorm1d(50, eps=1e-05, momentum=

In [60]:
# emb_szs = tabtext_db.get_emb_szs({})
# model = TabularModel(emba_szs, len(tabtext_db.cont_names), out_sz=tabtext_db.c, layers=[222,111],ps=[0.2,0.1])

In [55]:
# model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(165, 28)
    (2): Embedding(123, 24)
    (3): Embedding(4, 3)
    (4): Embedding(8, 5)
    (5): Embedding(8, 5)
    (6): Embedding(7, 5)
    (7): Embedding(5, 4)
    (8): Embedding(4, 3)
    (9): Embedding(4, 3)
    (10): Embedding(4, 3)
    (11): Embedding(4, 3)
    (12): Embedding(4, 3)
    (13): Embedding(14, 7)
    (14): Embedding(76, 18)
    (15): Embedding(3, 3)
    (16): Embedding(3, 3)
    (17): Embedding(3, 3)
    (18): Embedding(3, 3)
    (19): Embedding(3, 3)
    (20): Embedding(3, 3)
    (21): Embedding(3, 3)
    (22): Embedding(3, 3)
    (23): Embedding(3, 3)
    (24): Embedding(3, 3)
    (25): Embedding(3, 3)
    (26): Embedding(3, 3)
    (27): Embedding(3, 3)
    (28): Embedding(3, 3)
    (29): Embedding(3, 3)
    (30): Embedding(3, 3)
    (31): Embedding(3, 3)
    (32): Embedding(3, 3)
    (33): Embedding(3, 3)
    (34): Embedding(3, 3)
    (35): Embedding(3, 3)
    (36): Embedding(3, 3)
 

## Complete tabular text databunch

In [28]:
tabtext_db = get_tabulartext_databunch()

In [29]:
tabtext_db.show_batch(ds_type=DatasetType.Valid)

Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,State,m1_label_description,Is_Sterilized,Is_Free,IsMandarin,IsMalay,width_mean_na,metadata_metadata_color_pixelfrac_MEAN_na,image_size_std_na,width_std_na,sentiment_magnitude_na,height_mean_na,metadata_metadata_annots_score_SUM_na,metadata_metadata_crop_importance_MEAN_na,image_size_mean_na,metadata_metadata_crop_conf_MEAN_na,sentiment_document_magnitude_na,metadata_metadata_crop_importance_SUM_na,image_size_sum_na,height_sum_na,metadata_metadata_crop_conf_SUM_na,sentiment_score_na,metadata_metadata_color_score_MEAN_na,metadata_metadata_color_pixelfrac_SUM_na,height_std_na,metadata_metadata_color_score_SUM_na,metadata_metadata_annots_score_MEAN_na,width_sum_na,sentiment_document_score_na,TFIDF_Description_5,RescuerID_age_mean,general health,TFIDF_metadata_annots_top_desc_15,potential for mouthiness,width_mean,Fee,Breed1_m1_vertex_y_mean,TFIDF_metadata_annots_top_desc_11,RescuerID_Fee_Mean,Quantity,RescuerID_m1LabelScore_mean,TFIDF_Description_7,Breed1_Age_mean,TFIDF_metadata_annots_top_desc_14,TFIDF_sentiment_entities_10,tolerates hot weather,State_Pop,State_RescuerID_COUNT_sum,State_m1_label_score_mean,size,adapts well to apartment living,good for novice owners,drooling potential,TFIDF_sentiment_entities_14,metadata_metadata_color_pixelfrac_MEAN,intelligence,TFIDF_metadata_annots_top_desc_2,Breed1_RescuerID_nunique,easy to train,tendency to bark or howl,TFIDF_sentiment_entities_1,image_size_std,State_Density,TFIDF_metadata_annots_top_desc_13,width_std,m1_dominant_red,TFIDF_Description_9,sentiment_magnitude,height_mean,metadata_metadata_annots_score_SUM,metadata_metadata_crop_importance_MEAN,TFIDF_Description_2,TFIDF_Description_14,wanderlust potential,TFIDF_metadata_annots_top_desc_0,RescuerID_breed1_nunique,TFIDF_Description_3,image_size_mean,State_m1_vertex_y_mean,TFIDF_Description_6,State_Area,prey drive,metadata_metadata_crop_conf_MEAN,TFIDF_sentiment_entities_15,sentiment_document_magnitude,TFIDF_sentiment_entities_11,potential for playfulness,Age,TFIDF_metadata_annots_top_desc_10,Breed1_count,Breed1_m1_label_score_sum,m1_dominant_blue,metadata_metadata_crop_importance_SUM,TFIDF_sentiment_entities_6,State_count,PhotoAmt,TFIDF_sentiment_entities_8,image_size_sum,affectionate with family,TFIDF_metadata_annots_top_desc_9,easy to groom,pet friendly,friendly toward strangers,TFIDF_sentiment_entities_9,tendency to vocalize,incredibly kid friendly dogs,height_sum,m1_dominant_pixel_frac,TFIDF_sentiment_entities_5,RescuerID_m1VertexY_mean,RescuerID_IsFree_Mean,intensity,State_Age_mean,TFIDF_metadata_annots_top_desc_12,State_GDP,m1_vertex_y,metadata_metadata_crop_conf_SUM,TFIDF_Description_0,State_m1_label_score_sum,RescuerID_m1LabelScore_sum,RescuerID_Fee_Sum,TFIDF_metadata_annots_top_desc_5,TFIDF_sentiment_entities_4,TFIDF_sentiment_entities_12,TFIDF_metadata_annots_top_desc_7,potential for weight gain,sentiment_score,TFIDF_metadata_annots_top_desc_3,metadata_metadata_color_score_MEAN,metadata_metadata_color_pixelfrac_SUM,TFIDF_Description_13,TFIDF_sentiment_entities_13,height_std,State_RescuerID_nunique,State_RescuerID_COUNT_mean,TFIDF_Description_8,sensitivity level,m1_vertex_x,name_length,Breed1_RescuerID_COUNT_sum,TFIDF_sentiment_entities_3,metadata_metadata_color_score_SUM,TFIDF_Description_10,TFIDF_Description_1,TFIDF_sentiment_entities_0,TFIDF_Description_12,energy level,m1_bounding_confidence,m1_dominant_score,exercise needs,dog friendly,RescuerID_COUNT,m1_bounding_importance,TFIDF_metadata_annots_top_desc_4,amount of shedding,TFIDF_Description_11,metadata_metadata_annots_score_MEAN,tolerates being alone,TFIDF_sentiment_entities_7,Breed1_RescuerID_COUNT_mean,TFIDF_metadata_annots_top_desc_1,m1_dominant_green,TFIDF_metadata_annots_top_desc_8,width_sum,TFIDF_Description_4,kid friendly,tolerates cold weather,TFIDF_Description_15,TFIDF_metadata_annots_top_desc_6,VideoAmt,m1_label_score,Breed1_m1_label_score_mean,sentim

text,target
"xxbos(2) healthy(94) and(9) active(102) ,(10) feisty(2058) kitten(84) found(71) in(20) neighbours(1026) '(232) garden(786) .(8) xxmaj(4) not(46) sure(301) of(19) sex(1939)",1.0
"xxbos(2) xxmaj(4) hi(319) xxmaj(4) pet(149) xxmaj(4) lovers(572) !(43) xxmaj(4) this(49) is(14) my(32) first(231) posting(1619) and(9) i(16) need(123) help(161) !(43) 3(86) months(109) ago(261) we(36) befriended(5968) a(12) mother(142) stray(187) cat(50) with(23) 3(86) kittens(101) in(20) our(134) area(182) and(9) we(36) '(232) adopted(141) '(232) them(37) but(52) they(39) come(248) and(9) go(171) as(45) they(39) please(31) .(8) 2(62) months(109) ago(261) the(13) mother(142) gave(287) birth(244) to(11) a(12) litter(160) of(19) 7(286) cute(115) kittens(101) and(9) we(36) have(33) been(88) taking(367) care(79) of(19) the(13) mother(142) and(9) the(13) kittens(101) ever(625) since(195) in(20) our(134) home(25) ...(117) in(20) the(13) ground(2353) floor(1299) bathroom(4507) !(43) xxmaj(4) some(172) of(19) the(13) kittens(101) look(252) half(860) -(29) persian(804) with(23) long(157) hair(332) ,(10) xxunk(0) also(145) from(66) the(13) earlier(1557) stray(187) batch(2823) we(36) adopted(141) .(8) xxmaj(4) looking(82) for(15) good(60) caring(435) homes(378)",1.0
xxbos(2) to(11) be(27) spayed(200) on(53) /(56) 12(690) adorable(152) &(67),3.0
"xxbos(2) xxmaj(4) birth(244) xxmaj(4) date(1109) :(41) xxmaj(4) oct(1752) 30th(5133) xxmaj(4) kitty(309) 1(108) ,(10) xxmaj(4) xxunk(0) -female(6549) xxmaj(4) sangat(672) comel(1017) and(9) xxunk(0) .(8) xxmaj(4) kitty(309) 2(62) ,(10) xxmaj(4) tootsie(7369) -(29) xxmaj(4) male(143) xxmaj(4) badan(3157) putih(1662) ,(10) tapi(1206) ekor(784) hitam(2135) and(9) telinga(4065) xxunk(0) -(29) xxunk(0) ,(10) macam(2396) kucing(300) 3(86) color(516) .(8) xxmaj(4) kitty(309) 3(86) ,(10) xxmaj(4) bo(6550) -male(4787) xxmaj(4) corak(5134) badan(3157) 3(86) tompok(3443) ,(10) he(21) is(14) one(89) of(19) the(13) most(276) quite(333) and(9) xxunk(0) kitty(309) of(19) the(13) litter(160) tapi(1206) jantan(1640) .(8) xxmaj(4) kitty(309) 4(107) ,(10) xxmaj(4) xxunk(0) -(29) xxmaj(4) female(127) xxmaj(4) macam(2396) abang(4509) xxunk(0) ,(10) she(17) is(14) very(26) xxunk(0) and(9) sweet(179) ,(10) have(33) 1(108) big(289) spot(928) behind(723) her(18) back(168) and(9) 1(108) super(365) cute(115) spot(928) belakang(2916) telinga(4065) .(8) xxmaj(4) kitty(309) 5(146) ,(10) xxmaj(4) missy(2621) -(29) xxmaj(4) female(127) .(8) xxmaj(4) she(17) is(14) very(26) curious(674) and(9) adventurous(1139) .(8) xxmaj(4) very(26) lovable(579) ,(10) with(23) sweet(179) voice(1433)",4.0
"xxbos(2) xxmaj(4) garfield(2060) is(14) a(12) very(26) large(956) cat(50) .(8) xxmaj(4) needs(189) daily(580) grooming(939) .(8) xxmaj(4) he(21) has(58) been(88) groomed(1850) short(341) and(9) deflea(1753) .(8) xxmaj(4) very(26) good(60) wt(2759) other(112) cats(98) .(8) xxmaj(4) calm(610) cat(50) .(8) xxmaj(4) he(21) is(14) neutered(259) ,(10) vaccinated(191) and(9) dewormed(207) .(8) xxmaj(4) must(114) be(27) kept(353) indoors(379) .(8) xxmaj(4) needs(189) grooming(939) and(9) brushing(4276) as(45) his(68) fur(239) is(14) very(26) soft(478) and(9) thick(1332) .(8) xxmaj(4) if(35) you(24) are(22) interested(77) to(11) adopt(64) xxmaj(4) garfield(2060) ,(10) please(31) give(69) me(34) a(12) call(74) or(51) sent(546) me(34) an(106) email(250) .(8) thank(173) you(24)",2.0


In [41]:
x1,y1 = tabtext_db.one_batch(ds_type=DatasetType.Train)
y1,len(y1)

(tensor([2., 2., 4., 4., 4., 4., 4., 4., 1., 2., 4., 2., 3., 1., 2., 4., 2., 1.,
         1., 4., 4., 1., 4., 0., 2., 3., 3., 4., 3., 2., 1., 1., 2., 4., 3., 1.,
         4., 3., 4., 4., 1., 4., 1., 2., 4., 1., 3., 4., 4., 1., 4., 3., 3., 4.,
         2., 2., 3., 4., 4., 3., 4., 4., 4., 1.]), 64)

In [42]:
len(x1),len(x1[0]),len(x1[1]),x1[0].shape,x1[1].shape,x1[2].shape

(3, 64, 64, torch.Size([64, 42]), torch.Size([64, 145]), torch.Size([64, 765]))

In [32]:
x1[0][0] #cats

x1[1][0] #conts (normalized)

tensor([  2, 127,   1,   2,   6,   7,   1,   2,   2,   1,   1,   1,   1,   3,
         11,   2,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])

tensor([-3.8405e-01,  1.6226e+00, -4.5429e-01, -7.8942e-02, -3.6212e-01,
        -2.9247e-01, -2.7784e-01, -6.6113e-01, -4.0079e-01, -3.3711e-01,
        -3.9743e-01,  2.6175e-01, -1.0043e-01, -4.9071e-01, -1.4529e+00,
         1.8988e-01, -3.5761e-01,  8.3176e-01,  8.3500e-01,  1.1769e-01,
        -3.4273e-01, -3.4052e-01, -3.5297e-01, -3.0743e-01, -1.2816e-01,
         1.3650e+00, -4.4736e-01, -1.3967e-01, -6.2311e-01, -3.6190e-01,
        -3.4872e-01, -5.5261e-01, -6.3511e-01, -5.8299e-01,  1.1894e+00,
         2.3167e-01,  5.2292e-01, -3.0570e-01,  7.9755e-01, -1.0707e+00,
         1.4220e-02, -2.4799e-02, -2.1915e-01,  1.6981e-02, -3.5138e-01,
        -9.9167e-01, -4.0593e-01, -4.3433e-01, -1.2103e+00,  4.8278e-01,
         1.8928e-01,  2.1138e-01, -3.5382e-01, -1.0554e-01, -1.7935e-01,
         8.3119e-01, -2.3576e-01, -4.6717e-01,  2.0447e+00,  8.2465e-01,
        -8.5909e-01, -8.4936e-01,  8.7540e-01,  3.7276e-03,  3.2496e-02,
         8.0510e-01,  2.3973e-02, -1.7139e-01, -5.9

# Step by step to create TabularText databunch

## 1. create itemlist

itemlist: list of itembase, which contains independent variables

In [30]:
data_lm = load_data(path, 'tmp_lm.pkl', bs=32)

In [31]:
il = TabularTextList.from_df(train, cat_names, cont_names, txt_cols, vocab=data_lm.vocab, procs=procs, path=path)

In [82]:
print(f'CATS:\n{il.cat_names}')
# print(f'CONTS:\n{il.cont_names}')
print(f'TEXT COLS:\n{il.text_cols}')
print(f'PROCS:\n{il.procs}')

CATS:
['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'State', 'm1_label_description', 'Is_Sterilized', 'Is_Free', 'IsMandarin', 'IsMalay', 'image_size_std_na', 'sentiment_document_score_na', 'metadata_metadata_color_pixelfrac_MEAN_na', 'sentiment_document_magnitude_na', 'image_size_sum_na', 'metadata_metadata_crop_conf_MEAN_na', 'metadata_metadata_crop_conf_SUM_na', 'metadata_metadata_color_pixelfrac_SUM_na', 'sentiment_magnitude_na', 'height_std_na', 'metadata_metadata_color_score_SUM_na', 'metadata_metadata_color_score_MEAN_na', 'metadata_metadata_crop_importance_MEAN_na', 'metadata_metadata_annots_score_MEAN_na', 'width_sum_na', 'image_size_mean_na', 'width_std_na', 'height_mean_na', 'metadata_metadata_crop_importance_SUM_na', 'width_mean_na', 'metadata_metadata_annots_score_SUM_na', 'sentiment_score_na', 'height_sum_na']
TEXT COLS:
['Description']
PROCS:
[<class 'fastai.tabular.tran

In [86]:
il.get(0)

Type                                                                                         2
Breed1                                                                                     299
Breed2                                                                                       0
Gender                                                                                       1
Color1                                                                                       1
Color2                                                                                       7
Color3                                                                                       0
MaturitySize                                                                                 1
FurLength                                                                                    1
Vaccinated                                                                                   2
Dewormed                                          

## 2. Split train and validation

ItemLists: train ItemList and validation itemList

In [32]:
ils = il.split_by_idx(val_idxs[0])

In [48]:
len(ils.train), len(ils.valid), ils.path

(11939, 3051, PosixPath('/home/quantran/kwon/kaggle/new-comp/data'))

In [ ]:
#     data_clas = (TextList.from_df(X_train,path, vocab=data_lm.vocab)
#                  #grab all the text files in path
#                  .split_by_idx(val_idxs[0])
#                  #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
#                  .label_from_df(cols='AdoptionSpeed',label_cls=FloatList)
#                  #label them all with their folders
#                  .databunch(bs=32))


## 3: add y labels

add y label -> becomes label lists

this is essentially Pytorch Dataset, will be fed to model's forward pass

In [33]:
lls = ils.label_from_df(cols='AdoptionSpeed',label_cls=FloatList)

In [50]:
type(lls.valid)

fastai.data_block.LabelList

In [60]:
lls.valid.y[0]

FloatItem 1.0

In [59]:
lls.valid.x[0]

TabularText Type 2; Breed1 265; Breed2 0; Gender 2; Color1 6; Color2 0; Color3 0; MaturitySize 2; FurLength 2; Vaccinated 2; Dewormed 2; Sterilized 2; Health 1; State 41326; m1_label_description cat; Is_Sterilized False; Is_Free True; IsMandarin False; IsMalay False; image_size_std_na False; sentiment_document_score_na False; metadata_metadata_color_pixelfrac_MEAN_na False; sentiment_document_magnitude_na False; image_size_sum_na False; metadata_metadata_crop_conf_MEAN_na False; metadata_metadata_crop_conf_SUM_na False; metadata_metadata_color_pixelfrac_SUM_na False; sentiment_magnitude_na False; height_std_na False; metadata_metadata_color_score_SUM_na False; metadata_metadata_color_score_MEAN_na False; metadata_metadata_crop_importance_MEAN_na False; metadata_metadata_annots_score_MEAN_na False; width_sum_na False; image_size_mean_na False; width_std_na False; height_mean_na False; metadata_metadata_crop_importance_SUM_na False; width_mean_na False; metadata_metadata_annots_score_SUM

In [63]:
lls.valid.x[0].data

[tensor([  2, 127,   1,   2,   6,   1,   1,   2,   2,   2,   2,   2,   1,   3,
          11,   1,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]),
 tensor([ 0.2183,  0.4945,  0.4828,  0.1388, -0.3692, -0.3074,  0.3268, -0.3576,
         -0.1507,  0.0299, -0.3161, -0.3003, -0.4543,  0.0303, -0.3371, -0.7934,
         -0.3981, -0.4783,  2.7438, -0.0067,  1.4372, -0.8704,  0.7718, -0.2633,
         -0.4719,  0.8051,  0.3228, -1.0693, -0.3423,  0.6022, -0.5318, -0.3710,
         -0.3616, -0.4454, -2.4276, -0.0046, -0.3712, -0.8529,  0.4258, -0.4003,
         -0.6531,  0.3437,  1.3544, -0.5201, -0.8706, -0.7828,  0.8350,  0.0306,
         -0.3557,  0.7693, -0.4922,  0.3578, -0.1055, -0.9616,  0.2114, -0.4171,
          0.6080, -1.1070,  0.1319, -0.1105,  0.5871,  0.7101, -0.8303, -0.6207,
          0.2029,  0.0344,  0.7023, -0.3619, -0.6448, -0.3429,  0.5295,  0.6081,
         -0.6231, -0.5830, -0.47

In [65]:
lls.valid.x[0].text

"xxbos healthy and active , feisty kitten found in neighbours ' garden . xxmaj not sure of sex ."

In [62]:
lls.valid.x.text_ids[0]

array([   2,   94,    9,  102,   10, 2058,   84,   71,   20, 1026,  232,  786,    8,    4,   46,  301,   19, 1939,
          8])

In [67]:
len(lls.train.x.vocab.itos), len(lls.valid.x.vocab.itos)

(10131, 10131)

## 4: add test

In [34]:
lls = lls.add_test(TabularTextList.from_df(test, cat_names, cont_names, txt_cols,procs=procs, path=path))

In [70]:
lls.test.x[0].data

[tensor([  1, 164,   1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   1,   3,
          20,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]),
 tensor([ 1.2947e-01, -1.5042e+00,  4.5436e-01,  2.7206e-01,  9.8155e-01,
         -3.0743e-01, -2.2018e+00, -3.5761e-01, -1.5066e-01,  8.2639e-01,
          7.5734e-01, -5.0052e-01, -4.5429e-01, -1.6898e-02,  1.2211e+00,
          8.1120e-02,  2.1810e+00,  7.7454e-02,  3.0595e-01, -7.0714e-01,
         -8.8790e-01,  9.0379e-01,  3.5707e-01,  9.7715e-01, -4.7187e-01,
          6.5516e-01,  3.8508e-01, -1.4456e+00, -3.4227e-01, -2.6513e-01,
          3.4878e-01, -3.7101e-01, -3.6165e-01,  1.9814e-02, -3.4950e-02,
         -3.0038e+00, -3.7124e-01,  3.4839e-01, -3.9615e-01,  1.3585e+01,
          3.0051e+00, -8.5804e-01,  1.3302e-01, -5.2010e-01, -3.0337e-01,
          1.2564e+00,  4.7000e-01, -3.6701e-01, -3.5566e-01,  7.3454e-01,
         -2.0052e-01,

In [71]:
lls.test.x[0].text
lls.test.x.text_ids[0]

'xxbos xxmaj puppy is calm for a young dog , but he becomes very cheerful among people . xxmaj he likes being hugged and carried .'

array([   2,    4,  100,   14,  610,   15,   12,  279,   48,   10,   52,   21, 1485,   26,  868,  861,  156,    8,
          4,   21,  266,  169, 3208,    9, 1189,    8])

In [72]:
len(lls.test.x.vocab.itos)

10131

## 5: build databunch

collection of pytorch dataloaders

In [35]:
collate_fn = partial(mixed_tabular_pad_collate, pad_idx=1, pad_first=True)
tabtext_db = lls.databunch(bs=64,collate_fn=collate_fn, no_check=False)

In [107]:
x1,y1 = tabtext_db.one_batch(ds_type=DatasetType.Valid)

In [108]:
#label
y1,len(y1)

(tensor([1., 1., 3., 4., 2., 3., 4., 3., 4., 3., 3., 4., 4., 4., 2., 4., 3., 4.,
         1., 3., 3., 4., 3., 1., 2., 3., 2., 3., 2., 4., 4., 1., 2., 4., 3., 3.,
         2., 1., 4., 2., 2., 2., 4., 4., 4., 4., 1., 3., 2., 4., 1., 1., 2., 2.,
         2., 2., 2., 1., 4., 1., 2., 2., 2., 2.]), 64)

In [113]:
y1.shape

torch.Size([64])

In [88]:
x1,y1 = tabtext_db.one_batch(ds_type=DatasetType.Train)
# noted that each piece text (paragraph) has been patched to have equal size (= size of longest texts)
# this size can be different for each batch
x1[2][0].shape,x1[2][1].shape,x1[2][-1].shape

(torch.Size([461]), torch.Size([461]), torch.Size([461]))

In [84]:
x1,y1 = tabtext_db.one_batch(ds_type=DatasetType.Valid)

x1[2][0].shape,x1[2][1].shape,x1[2][-1].shape

(torch.Size([484]), torch.Size([484]), torch.Size([484]))

In [86]:
x1,y1 = tabtext_db.one_batch(ds_type=DatasetType.Test)

x1[2][0].shape,x1[2][1].shape,x1[2][-1].shape

(torch.Size([141]), torch.Size([141]), torch.Size([141]))

In [106]:
tabtext_db = get_tabulartext_databunch()

In [34]:
tabtext_db.show_batch()

Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,State,m1_label_description,Is_Sterilized,Is_Free,IsMandarin,IsMalay,metadata_metadata_crop_conf_SUM_na,sentiment_document_magnitude_na,height_sum_na,width_mean_na,metadata_metadata_annots_score_MEAN_na,image_size_std_na,height_std_na,image_size_sum_na,metadata_metadata_color_pixelfrac_MEAN_na,metadata_metadata_crop_importance_SUM_na,width_std_na,sentiment_document_score_na,width_sum_na,metadata_metadata_color_pixelfrac_SUM_na,sentiment_score_na,metadata_metadata_annots_score_SUM_na,height_mean_na,metadata_metadata_crop_importance_MEAN_na,sentiment_magnitude_na,metadata_metadata_color_score_MEAN_na,metadata_metadata_crop_conf_MEAN_na,metadata_metadata_color_score_SUM_na,image_size_mean_na,TFIDF_metadata_annots_top_desc_8,State_RescuerID_nunique,metadata_metadata_crop_conf_SUM,State_RescuerID_COUNT_mean,potential for playfulness,Breed1_RescuerID_COUNT_mean,TFIDF_Description_4,sentiment_document_magnitude,height_sum,affectionate with family,sensitivity level,Quantity,TFIDF_metadata_annots_top_desc_10,RescuerID_IsFree_Mean,potential for weight gain,kid friendly,State_m1_label_score_mean,TFIDF_metadata_annots_top_desc_7,easy to groom,Breed1_AdoptionSpeed_mean,width_mean,Age,TFIDF_sentiment_entities_15,metadata_metadata_annots_score_MEAN,TFIDF_Description_7,image_size_std,drooling potential,TFIDF_metadata_annots_top_desc_15,RescuerID_m1LabelScore_mean,RescuerID_COUNT,TFIDF_Description_11,TFIDF_Description_9,TFIDF_Description_2,TFIDF_sentiment_entities_1,State_GDP,height_std,tendency to bark or howl,m1_dominant_green,tolerates hot weather,TFIDF_metadata_annots_top_desc_13,State_m1_vertex_y_mean,State_m1_label_score_sum,image_size_sum,TFIDF_metadata_annots_top_desc_11,TFIDF_metadata_annots_top_desc_6,TFIDF_metadata_annots_top_desc_5,intelligence,Breed1_RescuerID_nunique,TFIDF_Description_14,TFIDF_sentiment_entities_12,State_RescuerID_COUNT_sum,PhotoAmt,TFIDF_Description_6,metadata_metadata_color_pixelfrac_MEAN,TFIDF_Description_15,metadata_metadata_crop_importance_SUM,TFIDF_Description_10,TFIDF_Description_12,TFIDF_sentiment_entities_6,width_std,TFIDF_sentiment_entities_7,good for novice owners,State_Age_mean,Breed1_count,TFIDF_Description_3,TFIDF_sentiment_entities_5,TFIDF_sentiment_entities_8,potential for mouthiness,State_Density,m1_dominant_score,TFIDF_metadata_annots_top_desc_1,RescuerID_m1LabelScore_sum,Breed1_Age_mean,sentiment_document_score,m1_label_score,width_sum,RescuerID_Fee_Sum,incredibly kid friendly dogs,TFIDF_metadata_annots_top_desc_2,TFIDF_sentiment_entities_9,TFIDF_Description_0,RescuerID_m1VertexY_mean,metadata_metadata_color_pixelfrac_SUM,TFIDF_metadata_annots_top_desc_4,RescuerID_breed1_nunique,TFIDF_sentiment_entities_4,general health,RescuerID_Fee_Mean,State_count,m1_dominant_pixel_frac,TFIDF_metadata_annots_top_desc_9,sentiment_score,TFIDF_Description_1,TFIDF_sentiment_entities_10,adapts well to apartment living,metadata_metadata_annots_score_SUM,wanderlust potential,height_mean,metadata_metadata_crop_importance_MEAN,State_Area,sentiment_magnitude,m1_bounding_confidence,m1_bounding_importance,RescuerID_age_mean,metadata_metadata_color_score_MEAN,amount of shedding,TFIDF_sentiment_entities_0,State_Pop,Fee,energy level,metadata_metadata_crop_conf_MEAN,TFIDF_Description_8,size,name_length,Breed1_m1_vertex_y_mean,Breed1_RescuerID_COUNT_sum,intensity,TFIDF_sentiment_entities_13,easy to train,m1_vertex_y,TFIDF_sentiment_entities_3,TFIDF_metadata_annots_top_desc_0,Breed1_m1_label_score_mean,TFIDF_sentiment_entities_14,tolerates cold weather,TFIDF_metadata_annots_top_desc_14,TFIDF_metadata_annots_top_desc_3,friendly toward strangers,TFIDF_metadata_annots_top_desc_12,tolerates being alone,exercise needs,m1_dominant_red,VideoAmt,dog friendly,m1_dominant_blue,TFIDF_Description_5,metadata_metadata_color_score_SUM,prey drive,Breed1_m1_label_score_sum,TFIDF_Description_13,TFIDF_sentiment_entities_2,tendency to vocali

text,target
"xxbos(2) xxmaj(4) maya(2040) is(14) the(13) queen(2592) in(20) the(13) house(96) .(8) xxmaj(4) she(17) 's(55) beautiful(222) ,(10) sweet(179) and(9) looks(421) blur(3844) at(30) times(456) .(8) xxmaj(4) when(78) you(24) hug(1466) her(18) ,(10) it(59) 's(55) like(95) hugging(5395) a(12) soft(478) feathered(9375) xxunk(0) .(8) xxmaj(4) always(263) ready(262) for(15) hugs(1683) and(9) kisses(1867) ,(10) this(49) lady(593) xxunk(0) in(20) tickling(9298) on(53) her(18) stomach(2471) and(9) neck(1385) .(8) xxmaj(4) she(17) will(38) lean(2917) over(324) to(11) ask(630) for(15) a(12) kiss(2635) on(53) her(18) nose(1081) and(9) seldom(1681) make(230) any(148) noise(1135) .(8) xxmaj(4) feeds(2046) only(93) on(53) dry(551) kibbles(401) .(8) xxmaj(4) easy(474) maintenance(2114) .(8) p(782) /(56) s(470) :(41) xxmaj(4) no(90) cage(322) confinement(2099) please(31) ,(10) except(654) in(20) incidences(4072) of(19) illness(1492) or(51) injuries(1510) .(8)",1.0
xxbos(2) xxmaj(4) he(21) was(40) nearly(1649) became(1740) a(12) road(346) kill(1941) before(197) me(34) and(9) my(32) wife(1272) and(9) also(145) some(172) xxmaj(4) good(60) xxmaj(4) samaritans(4307) help(161) to(11) rescue(357) him(42) .(8) xxmaj(4) very(26) affectionate(338) little(124) guy(846) !(43),3.0
"xxbos(2) xxmaj(4) these(140) kittens(101) were(151) born(264) at(30) my(32) grandmother(4719) 's(55) yard(1595) ,(10) last(358) week(325) i(16) found(71) the(13) mother(142) dead(1514) ,(10) maybe(955) hit(706) by(61) a(12) car(272) around(125) 5(146) minutes(2732) before(197) i(16) saw(596) her(18) .(8) xxmaj(4) the(13) kittens(101) are(22) almost(476) 2(62) months(109) old(65) .(8) xxmaj(4) they(39) know(219) how(273) to(11) drink(1094) milk(457) by(61) themselves(2035) .(8) xxmaj(4) just(126) need(123) to(11) clean(405) them(37) up(87) every(486) time(113) they(39) poop(1614) .(8) xxmaj(4) really(163) need(123) someone(165) to(11) adopt(64) as(45) i(16) already(159) have(33) 7(286) cats(98) at(30) home(25) and(9) the(13) milking(5380) mother(142) is(14) not(46) keen(1063) on(53) giving(400) them(37) milk(457) .(8)",1.0
"xxbos(2) xxmaj(4) found(71) him(42) a(12) drain(443) 2(62) weeks(180) ago(261) .(8) xxmaj(4) super(365) playful(76) and(9) gets(430) along(283) great(302) with(23) other(112) cats(98) .(8) xxmaj(4) very(26) smart(314) and(9) never(372) messes(4199) up(87) the(13) house(96) .(8) xxmaj(4) he(21) also(145) knows(431) how(273) to(11) use(464) the(13) litter(160) box(271) and(9) is(14) not(46) used(429) to(11) be(27) alone(411) ((44) especially(749) at(30) xxunk(0) u(209) adopt(64) him(42) and(9) find(135) out(155) that(57) he(21) is(14) not(46) really(163) the(13) kitten(84) u(209) want(196) ,(10) u(209) may(249) always(263) return(517) him(42) back(168) to(11) me(34) .(8) xxmaj(4) pls(158) email(250) me(34) if(35) interested(77) :)(183)",3.0
"xxbos(2) 20(812) xxmaj(4) april(791) update(480) :(41) minpin(6178) is(14) on(53) trial(3401) adoption(54) minpin(6178) was(40) found(71) on(53) a(12) busy(578) road(346) and(9) is(14) being(169) put(310) up(87) for(15) adoption(54) since(195) no(90) one(89) claimed(2650) her(18) .(8) xxmaj(4) she(17) 's(55) now(85) believed(1486) to(11) be(27) abandoned(208) due(217) to(11) her(18) cataract(4259) .(8) xxmaj(4) due(217) to(11) her(18) early(1410) cataract(4259) which(213) is(14) probably(692) xxunk(0) inherited(5286) ,(10) she(17) can(47) only(93) see(293) xxunk(0) from(66) her(18) right(330) eye(504) although(769) she(17) has(58) good(60) vision(7216) from(66) her(18) left(254) eye(504) .(8) xxmaj(4) this(49) would(116) caused(1805) her(18) to(11) panic(4944) and(9) start(945) protecting(5532) herself(824) by(61) xxunk(0) if(35) there(130) 's(55) any(148) sudden(4807) movement(3658) .(8) xxmaj(4) however(364) ,(10) when(78) she(17) knows(431) you(24) ,(10) get(150) used(429) to(11) your(111) voice(1433) ,(10) she(17) will(38) be(27) as(45) adorable(152) as(45) a(12) kitten(84) .(8) xxmaj(4) in(20) addition(1503) ,(

# Compare to tabular data

In [53]:
# itemlist: list of itembase, which contains independent variables
data = TabularList.from_df(train, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)

In [54]:
type(data)

fastai.tabular.data.TabularList

In [40]:
item_lists = data.split_by_idx(val_idxs[0]) # train ItemList and validation itemList

In [42]:
type(item_lists.valid) # there is no label yet (e.g. item_lists.valid.y)

fastai.tabular.data.TabularList

In [48]:
label_lists = item_lists.label_from_df(cols=dep_var,label_cls = FloatList) # add y label -> label lists

In [49]:
type(label_lists.valid) # this is essentially Pytorch Dataset, will be fed to model's forward pass

fastai.data_block.LabelList

In [50]:
print(type(label_lists.valid.y))
print(type(label_lists.valid.x))

<class 'fastai.data_block.FloatList'>
<class 'fastai.tabular.data.TabularList'>


In [55]:
data_tab = get_databunch()

In [32]:
data_tab.show_batch(ds_type=DatasetType.Valid)

Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,State,m1_label_description,Is_Sterilized,Is_Free,IsMandarin,IsMalay,image_size_mean_na,metadata_metadata_color_score_MEAN_na,metadata_metadata_crop_conf_SUM_na,image_size_std_na,metadata_metadata_color_pixelfrac_MEAN_na,metadata_metadata_annots_score_MEAN_na,metadata_metadata_crop_importance_MEAN_na,image_size_sum_na,height_sum_na,sentiment_score_na,sentiment_document_score_na,metadata_metadata_crop_conf_MEAN_na,metadata_metadata_color_pixelfrac_SUM_na,metadata_metadata_color_score_SUM_na,metadata_metadata_crop_importance_SUM_na,height_std_na,width_sum_na,metadata_metadata_annots_score_SUM_na,width_mean_na,sentiment_document_magnitude_na,height_mean_na,sentiment_magnitude_na,width_std_na,image_size_mean,State_Pop,Quantity,TFIDF_metadata_annots_top_desc_15,State_count,RescuerID_age_mean,m1_vertex_x,TFIDF_Description_3,TFIDF_metadata_annots_top_desc_11,RescuerID_m1LabelScore_sum,metadata_metadata_color_score_MEAN,State_m1_vertex_y_mean,Breed1_m1_vertex_y_mean,TFIDF_sentiment_entities_3,potential for weight gain,metadata_metadata_crop_conf_SUM,friendly toward strangers,affectionate with family,TFIDF_metadata_annots_top_desc_1,TFIDF_sentiment_entities_1,TFIDF_Description_8,Breed1_AdoptionSpeed_mean,intensity,TFIDF_sentiment_entities_9,image_size_std,State_Density,TFIDF_Description_2,State_RescuerID_COUNT_sum,TFIDF_sentiment_entities_12,energy level,incredibly kid friendly dogs,easy to train,TFIDF_Description_0,metadata_metadata_color_pixelfrac_MEAN,TFIDF_metadata_annots_top_desc_12,metadata_metadata_annots_score_MEAN,TFIDF_Description_7,metadata_metadata_crop_importance_MEAN,TFIDF_metadata_annots_top_desc_5,Breed1_RescuerID_COUNT_mean,image_size_sum,TFIDF_Description_5,PhotoAmt,TFIDF_metadata_annots_top_desc_10,tolerates cold weather,TFIDF_metadata_annots_top_desc_3,Age,State_GDP,height_sum,State_Area,Breed1_Age_mean,Breed1_RescuerID_nunique,Breed1_count,TFIDF_sentiment_entities_13,m1_bounding_confidence,tolerates being alone,TFIDF_sentiment_entities_8,TFIDF_sentiment_entities_15,TFIDF_Description_1,good for novice owners,sentiment_score,sentiment_document_score,TFIDF_Description_4,TFIDF_Description_15,TFIDF_Description_14,TFIDF_metadata_annots_top_desc_13,adapts well to apartment living,metadata_metadata_crop_conf_MEAN,TFIDF_metadata_annots_top_desc_8,exercise needs,TFIDF_metadata_annots_top_desc_7,tendency to vocalize,tendency to bark or howl,RescuerID_m1VertexY_mean,name_length,TFIDF_metadata_annots_top_desc_14,TFIDF_metadata_annots_top_desc_6,Breed1_RescuerID_COUNT_sum,intelligence,TFIDF_Description_9,TFIDF_Description_12,TFIDF_sentiment_entities_2,TFIDF_sentiment_entities_7,metadata_metadata_color_pixelfrac_SUM,wanderlust potential,metadata_metadata_color_score_SUM,RescuerID_Fee_Mean,State_m1_label_score_mean,State_m1_label_score_sum,prey drive,RescuerID_breed1_nunique,TFIDF_sentiment_entities_11,tolerates hot weather,size,m1_dominant_blue,TFIDF_sentiment_entities_6,TFIDF_sentiment_entities_5,State_RescuerID_nunique,m1_dominant_red,metadata_metadata_crop_importance_SUM,height_std,Fee,TFIDF_sentiment_entities_10,width_sum,TFIDF_metadata_annots_top_desc_2,State_Age_mean,TFIDF_Description_10,m1_dominant_green,potential for playfulness,RescuerID_m1LabelScore_mean,general health,sensitivity level,metadata_metadata_annots_score_SUM,width_mean,sentiment_document_magnitude,Breed1_m1_label_score_mean,m1_vertex_y,m1_bounding_importance,height_mean,State_RescuerID_COUNT_mean,TFIDF_Description_6,TFIDF_metadata_annots_top_desc_0,pet friendly,amount of shedding,potential for mouthiness,kid friendly,RescuerID_Fee_Sum,dog friendly,sentiment_magnitude,easy to groom,Breed1_m1_label_score_sum,VideoAmt,TFIDF_sentiment_entities_0,RescuerID_COUNT,m1_dominant_score,TFIDF_sentiment_entities_14,TFIDF_metadata_annots_top_desc_9,width_std,TFIDF_Description_13,RescuerID_IsFree_Mean,m1_dominant_pixel_frac,m1_label_score,TFIDF_sentiment_entities_4,T

In [33]:
data_tab.show_batch(ds_type=DatasetType.Test)

Type,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,State,m1_label_description,Is_Sterilized,Is_Free,IsMandarin,IsMalay,image_size_mean_na,metadata_metadata_color_score_MEAN_na,metadata_metadata_crop_conf_SUM_na,image_size_std_na,metadata_metadata_color_pixelfrac_MEAN_na,metadata_metadata_annots_score_MEAN_na,metadata_metadata_crop_importance_MEAN_na,image_size_sum_na,height_sum_na,sentiment_score_na,sentiment_document_score_na,metadata_metadata_crop_conf_MEAN_na,metadata_metadata_color_pixelfrac_SUM_na,metadata_metadata_color_score_SUM_na,metadata_metadata_crop_importance_SUM_na,height_std_na,width_sum_na,metadata_metadata_annots_score_SUM_na,width_mean_na,sentiment_document_magnitude_na,height_mean_na,sentiment_magnitude_na,width_std_na,image_size_mean,State_Pop,Quantity,TFIDF_metadata_annots_top_desc_15,State_count,RescuerID_age_mean,m1_vertex_x,TFIDF_Description_3,TFIDF_metadata_annots_top_desc_11,RescuerID_m1LabelScore_sum,metadata_metadata_color_score_MEAN,State_m1_vertex_y_mean,Breed1_m1_vertex_y_mean,TFIDF_sentiment_entities_3,potential for weight gain,metadata_metadata_crop_conf_SUM,friendly toward strangers,affectionate with family,TFIDF_metadata_annots_top_desc_1,TFIDF_sentiment_entities_1,TFIDF_Description_8,Breed1_AdoptionSpeed_mean,intensity,TFIDF_sentiment_entities_9,image_size_std,State_Density,TFIDF_Description_2,State_RescuerID_COUNT_sum,TFIDF_sentiment_entities_12,energy level,incredibly kid friendly dogs,easy to train,TFIDF_Description_0,metadata_metadata_color_pixelfrac_MEAN,TFIDF_metadata_annots_top_desc_12,metadata_metadata_annots_score_MEAN,TFIDF_Description_7,metadata_metadata_crop_importance_MEAN,TFIDF_metadata_annots_top_desc_5,Breed1_RescuerID_COUNT_mean,image_size_sum,TFIDF_Description_5,PhotoAmt,TFIDF_metadata_annots_top_desc_10,tolerates cold weather,TFIDF_metadata_annots_top_desc_3,Age,State_GDP,height_sum,State_Area,Breed1_Age_mean,Breed1_RescuerID_nunique,Breed1_count,TFIDF_sentiment_entities_13,m1_bounding_confidence,tolerates being alone,TFIDF_sentiment_entities_8,TFIDF_sentiment_entities_15,TFIDF_Description_1,good for novice owners,sentiment_score,sentiment_document_score,TFIDF_Description_4,TFIDF_Description_15,TFIDF_Description_14,TFIDF_metadata_annots_top_desc_13,adapts well to apartment living,metadata_metadata_crop_conf_MEAN,TFIDF_metadata_annots_top_desc_8,exercise needs,TFIDF_metadata_annots_top_desc_7,tendency to vocalize,tendency to bark or howl,RescuerID_m1VertexY_mean,name_length,TFIDF_metadata_annots_top_desc_14,TFIDF_metadata_annots_top_desc_6,Breed1_RescuerID_COUNT_sum,intelligence,TFIDF_Description_9,TFIDF_Description_12,TFIDF_sentiment_entities_2,TFIDF_sentiment_entities_7,metadata_metadata_color_pixelfrac_SUM,wanderlust potential,metadata_metadata_color_score_SUM,RescuerID_Fee_Mean,State_m1_label_score_mean,State_m1_label_score_sum,prey drive,RescuerID_breed1_nunique,TFIDF_sentiment_entities_11,tolerates hot weather,size,m1_dominant_blue,TFIDF_sentiment_entities_6,TFIDF_sentiment_entities_5,State_RescuerID_nunique,m1_dominant_red,metadata_metadata_crop_importance_SUM,height_std,Fee,TFIDF_sentiment_entities_10,width_sum,TFIDF_metadata_annots_top_desc_2,State_Age_mean,TFIDF_Description_10,m1_dominant_green,potential for playfulness,RescuerID_m1LabelScore_mean,general health,sensitivity level,metadata_metadata_annots_score_SUM,width_mean,sentiment_document_magnitude,Breed1_m1_label_score_mean,m1_vertex_y,m1_bounding_importance,height_mean,State_RescuerID_COUNT_mean,TFIDF_Description_6,TFIDF_metadata_annots_top_desc_0,pet friendly,amount of shedding,potential for mouthiness,kid friendly,RescuerID_Fee_Sum,dog friendly,sentiment_magnitude,easy to groom,Breed1_m1_label_score_sum,VideoAmt,TFIDF_sentiment_entities_0,RescuerID_COUNT,m1_dominant_score,TFIDF_sentiment_entities_14,TFIDF_metadata_annots_top_desc_9,width_std,TFIDF_Description_13,RescuerID_IsFree_Mean,m1_dominant_pixel_frac,m1_label_score,TFIDF_sentiment_entities_4,T

In [56]:
x1,y1 = data_tab.one_batch(ds_type=DatasetType.Train)

In [57]:
#label
y1,len(y1)

# each batch contains 2 columns: cats, conts
len(x1),len(x1[0]),len(x1[1]),x1[0].shape,x1[1].shape

x1[0][0] #cats

x1[1][0] #conts (normalized)

(tensor([3., 2., 2., 3., 3., 1., 2., 3., 1., 2., 2., 4., 4., 4., 4., 0., 4., 1.,
         2., 4., 4., 1., 2., 2., 2., 3., 2., 4., 2., 4., 4., 1., 2., 2., 2., 2.,
         2., 4., 4., 4., 1., 0., 4., 1., 4., 4., 3., 1., 4., 3., 4., 3., 2., 2.,
         1., 2., 2., 4., 2., 3., 2., 4., 4., 0.]), 64)

(2, 64, 64, torch.Size([64, 42]), torch.Size([64, 145]))

tensor([  2, 156,   1,   2,   2,   7,   1,   2,   1,   2,   1,   2,   1,   5,
         11,   1,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])

tensor([-0.8772,  0.3806, -0.4752, -0.2254, -0.4003, -0.6432, -0.1776, -1.1442,
        -0.2383, -0.2890, -0.1921,  0.0399, -0.1515, -0.1222, -1.7294, -0.6351,
         0.0047, -0.3074, -0.3371, -0.1649, -0.0869, -0.3537,  1.7047, -0.3509,
        -0.3481,  0.0039, -0.2233, -0.2274, -1.2558,  0.0348, -0.2906, -0.2756,
        -0.0248,  2.0888, -0.4779,  0.2517, -0.7411,  0.1933, -1.0589, -0.2842,
         0.7111,  0.7377, -0.2677,  0.2045, -0.1867, -0.2487,  0.1403,  0.6815,
        -0.0841, -0.8285, -0.4189,  0.2257, -0.0235,  0.5070, -0.3538,  0.2559,
        -0.8663, -0.1055, -0.1869, -0.6207,  2.9088, -1.1089, -0.4543, -0.4417,
        -0.1507, -0.6268, -0.4719, -0.0248, -0.3423, -0.9304, -0.4091, -1.5054,
        -0.3657, -0.3514, -0.1035, -0.4544, -0.0917, -1.6352, -0.3712,  2.0422,
         0.1400, -0.5742, -0.5056, -0.8795,  0.4945, -0.3621, -1.2210,  0.4495,
         0.1493,  0.1636, -0.3557, -0.3619, -1.0004, -1.7584,  0.8247, -0.3427,
        -0.3074,  1.5305, -0.4672, -0.80

# Compare to text databunch

In [41]:
txt_data = get_text_databunch(bs=64)

In [37]:
type(txt_data)

fastai.text.data.TextClasDataBunch

In [38]:
type(txt_data.valid_ds.x)

fastai.text.data.TextList

In [44]:
txt_data.show_batch(ds_type = DatasetType.Valid)

text,target
xxbos xxmaj for more pics go here : 9th xxmaj april edit : i have xxunk the adoption fee to xxup rm to include the fee i paid her breeder . xxmaj the reason i am now making reimbursement of the breeder 's fee compulsory is to help out xxmaj malaysian xxmaj dogs xxmaj deserve xxmaj better ( xxup mddb ) - they are currently in xxunk financial straits (,3.0
xxbos xxmaj profile xxmaj xxunk : xxmaj name : xxmaj mama xxmaj kin xxmaj age : xxmaj adult cat . xxmaj probably 1 year and half . xxmaj condition : xxmaj healthy and xxmaj pregnant ( xxmaj probably will deliver in early xxmaj may ) . xxmaj believed that this is not the first time she is pregnant . xxmaj characteristics : xxmaj adorable . xxmaj soft . xxmaj very,2.0
"xxbos xxmaj my people kept many of us . xxmaj we were all semi long haired . i loved them because they were kind and fed us well . xxmaj the small ones would sometimes make us toys from rolled up news paper and bits of string . xxmaj they also let us out of the cage we lived in and i enjoyed running and chasing , climbing and rolling",4.0
"xxbos [ xxmaj adopted 17 xxmaj november by xxmaj xxunk t ] "" i 'm a cute young standard xxmaj english bull terrier with lots of love to give . i love snuggling up for cuddles and belly rubs . xxmaj canine buddies make me super happy , i really love to play ! xxmaj like most of my xxunk , i 'm a xxunk xxunk ready to xxunk you",2.0
xxbos xxmaj please send an sms before calling if possible because i might not pick up your calls when i 'm busy and might not call back unknown missed calls . xxmaj or you can send e - mail . xxmaj these two kittens are xxmaj beel and xxunk . xxmaj they are kids of on of the stray cats we feed . xxmaj we picked them up from the,2.0


In [40]:
type(txt_data.valid_ds.x[0])

fastai.text.data.Text

In [41]:
txt_data.valid_ds.x[0]

Text xxbos healthy and active , feisty kitten found in neighbours ' garden . xxmaj not sure of sex .

In [48]:
txt_data.valid_ds.x[0].data

array([   2,   94,    9,  102,   10, 2058,   84,   71,   20, 1026,  232,  786,    8,    4,   46,  301,   19, 1939,
          8])

In [42]:
x1,y1 = txt_data.one_batch(ds_type=DatasetType.Train)
#label
y1,len(y1)
# each batch contains 1 column: text_ids

x1.shape

(tensor([4., 3., 2., 1., 2., 3., 3., 4., 4., 2., 4., 3., 4., 2., 1., 1., 4., 4.,
         4., 3., 2., 2., 4., 1., 3., 1., 1., 2., 3., 1., 1., 3., 3., 2., 4., 2.,
         4., 2., 1., 2., 4., 4., 4., 2., 4., 4., 0., 2., 1., 4., 4., 4., 4., 0.,
         4., 4., 4., 0., 2., 3., 1., 2., 2., 4.]), 64)

torch.Size([64, 1560])

In [43]:
x1[0][-50:]

tensor([2693,   13,  149,   44,    9,  132,   28,   19,  148, 1888, 1923,    8,
           4,   36,   75,   72,   19,  134, 1192,    9,  616,   37,   95,  134,
         290,  465,   10,   52,   36,  145, 3286,   37,  382, 7640,    8,    4,
          36,  815,   95,   24,   11,  352,   19,   37,   45,  111,  290,  465,
         139,    8])

In [95]:
x1,y1 = txt_data.one_batch(ds_type=DatasetType.Valid)
#label
y1,len(y1)
# each batch contains 1 column: text_ids

x1.shape

(tensor([3., 2., 4., 2., 2., 4., 2., 3., 4., 3., 1., 4., 3., 1., 2., 2., 3., 1.,
         3., 3., 4., 4., 2., 1., 4., 1., 4., 4., 3., 3., 1., 4.]), 32)

32

torch.Size([32, 1347])

In [98]:
x1,y1 = txt_data.one_batch(ds_type=DatasetType.Test)
#label
y1,len(y1)
# each batch contains 1 column: text_ids

x1.shape

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]), 32)

32

torch.Size([32, 4])